In [1]:
### packages
import cv2
import numpy as np
import time
import tkinter as tk
import easygui
import dlib
import uuid
import os
from matplotlib import pyplot as plt
from distutils.sysconfig import get_python_lib

In [2]:
### 캐스케이드 파일
cascade_file =  'haarcascade_frontalface_alt2.xml'
cascade = cv2.CascadeClassifier(cascade_file)

In [3]:
# 랜드마크
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat') ### face landmark
list_landmarks = []

In [4]:
# 얼굴 저장 디렉토리 생성
face_dir_all = "/Users/PDG/Desktop/2023 1학기(2학년)/2023 imganal/영상처리 프로젝트/face/all" ## 전체화면 캡쳐 디렉토리
face_dir = "/Users/PDG/Desktop/2023 1학기(2학년)/2023 imganal/영상처리 프로젝트/face"        ## 얼굴부분 캡쳐 디렉토리
os.makedirs(face_dir, exist_ok=True)

# 얼굴 검출 (고유번호 부착)

##### (고유번호를 부착하여 파일 인덱싱을 통해 편하게 중복검사를 하려 했지만 번호를 부착하는 것이 의미가 없었음)

In [ ]:
cap = cv2.VideoCapture(0)
# 시간 추적을 위한 변수 초기화
start_time = time.time()
save_interval = 1  # 저장 간격 (초)
file_counter = 1  # 파일 순서를 추적 변수
file_counter_all = 1 # all_파일 순서를 추적 변수

# 저장할 이미지의 고정된 크기
fixed_width = 1200
fixed_height = 1200

# 얼굴 영역을 넓게 할 값
extra_width = 100
extra_height = 100

while True:
    # 프레임 읽기
    ret, frame = cap.read()

    if ret:
        # 키보드 입력
        k = cv2.waitKey(1)

        # 종료
        if k == ord("q"):
            break
        
        # 그레이스케일로 변환
        img_gs = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # 얼굴 검출
        face_list = cascade.detectMultiScale(img_gs, minSize=(30, 30))
        
        # 얼굴 좌표 확인
        if len(face_list) > 0:
            current_time = time.time()
            elapsed_time = current_time - start_time
            
            # 일정 시간이 경과하면 얼굴 저장
            if elapsed_time >= save_interval:
                for face in face_list:
                    x, y, w, h = face
                    x_start = max(0, x - extra_width)
                    y_start = max(0, y - extra_height)
                    x_end = min(frame.shape[1], x + w + extra_width)
                    y_end = min(frame.shape[0], y + h + extra_height)
                    face_img = frame[y_start:y_end, x_start:x_end]  # 좀 더 넓은 얼굴 영역 추출
                    # 얼굴 영역 크기 조정
                    resized_face_img = cv2.resize(face_img, (fixed_width, fixed_height))
                    # 이하 저장 및 처리 코드
                    face_filename = f"{file_counter}){time.strftime('%Y%m%d_%H%M%S', time.localtime())}.jpg"  # 파일 이름 생성
                    face_filepath = os.path.join(face_dir, face_filename)  # 저장 경로 생성
                    cv2.imwrite(face_filepath, resized_face_img)  # 얼굴 이미지 저장
                    file_counter += 1  # 파일 순서 증가
                    
            if elapsed_time >= save_interval:
                for face in face_list:
                    face_img = frame  # 전체 화면 저장
                    # 전체 화면 크기 조정
                    resized_face_img = cv2.resize(face_img, (fixed_width, fixed_height))
                    # 이하 저장 및 처리 코드
                    face_filename = f"{file_counter_all}){time.strftime('%Y%m%d_%H%M%S_all', time.localtime())}.jpg"  # 파일 이름 생성
                    face_filepath = os.path.join(face_dir_all, face_filename)  # 저장 경로 생성
                    cv2.imwrite(face_filepath, resized_face_img)  # 얼굴 이미지 저장
                    #file_counter_all += 1  # 파일 순서 증가

                    # 색상 red
                    color = (0, 0, 255)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), color, thickness=3)
                
                # 시간 업데이트
                start_time = current_time
        
        # 프레임 출력
        cv2.imshow("VideoFrame", frame)
    else:
        print("Can't receive frame. Exiting...")
        break

# 캡처 해제 및 윈도우 종료
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


# 얼굴중복검사 (유클리디안 거리)

##### 임계값은 얼굴 특징 벡터 간의 유클리디안 거리를 기준으로 중복 여부를 판단하는 역할. 
##### 유클리디안 거리는 두 벡터 간의 거리를 측정하는 방법 중 하나로, 벡터 간의 유사도를 측정하는 데 사용. 
##### 임계값은 유클리디안 거리가 얼마 이하일 때 두 얼굴을 중복으로 판단할지를 결정.

### "임계값을 낮게 설정할수록 두 얼굴이 더 유사하게 인식되어 중복으로 판단되는 경우가 상승." 
### "반대로 임계값을 높게 설정할수록 두 얼굴이 덜 유사하게 인식되어 중복으로 판단되는 경우가 줄어듬."

##### 적절한 임계값은 데이터셋 및 애플리케이션의 요구 사항에 따라 다를 수 있음. 
##### 일반적으로는 실험과 조정을 통해 적절한 임계값을 결정. 
##### 데이터셋에 있는 얼굴들 간의 유사도를 고려하여 임계값을 조정하면 중복 검사의 정확도를 높일 수 있다.

### 유클리디안 거리 함수 지정

In [6]:
# 중복 얼굴 검사 함수 (유클리디안 거리)
def compare_face_features(face_feature1, face_feature2, threshold):
    # 얼굴 특징 벡터의 유클리디안 거리 계산
    distance = np.linalg.norm(face_feature1 - face_feature2)
    
    # 거리가 임계값보다 작으면 중복으로 판단
    if distance < threshold:
        return True
    else:
        return False

### 중복검사 시작

In [139]:
detector = dlib.get_frontal_face_detector()  # 얼굴 검출기 및 랜드마크 모델 초기화
predictor_path = "shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(predictor_path)

############################################################################################

## 개인 설정 필요! 
saved_face_features = []                     # 저장된 얼굴 특징 벡터 리스트 초기화
image_dir = "/Users/PDG/Desktop/face"    # 경로 설정 "all" 일 경우 얼굴 부분이 아닌 전체화면에서 인식 

############################################################################################

threshold = 0.3                  # 임계값 설정 (gpt: 보편적으로 0.5~1.0 사이)                                                              # 임계값 설정
## 임계값을 0.1 ~ 100.0 까지 약 40 번 돌려보았으나 변경점은 없었음 
## 임베딩 모델을 사용한 경우가 얼굴 중복검사적중률이 더 높았음 
############################################################################################
                                                             
for filename in os.listdir(image_dir):                        # 이미지 디렉토리에서 이미지 파일들을 읽어들여 얼굴 검출 및 중복 검사시작 
    if filename.endswith(".jpg"):
        image_path = os.path.join(image_dir, filename)
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
                                                              # 얼굴 검출
        faces = detector(gray)
        
        for face in faces:
            landmarks = predictor(gray, face)
            face_feature = np.array([[p.x, p.y] for p in landmarks.parts()])
            
                                                              # 중복 검사
            is_duplicate = False
            for saved_face_feature in saved_face_features:
                
                                                              # 기존에 저장된 얼굴 특징과 비교
                if compare_face_features(face_feature, saved_face_feature, threshold=10.0):
                    is_duplicate = True
                    break
            
            
            if not is_duplicate:
                saved_face_features.append(face_feature)
                                                              # 중복되지 않은 얼굴 이미지 출력
                cv2.imshow("Face Image", image)
                cv2.waitKey(0)
                
############################################################################################
                                                                                            
num_unique_faces = len(saved_face_features)                   # 중복되지 않은 얼굴들의 개수 출력
print("검출된 인원: ", num_unique_faces,"명")

############################################################################################
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


검출된 인원:  4 명


-1

# --------------------------------------------------------

# 얼굴 중복검사 (임베딩 모델사용)
##### 랜드마크 추출 후 임베딩모델로 얼굴 특징 벡터 계산 후  
##### 결국에 비교는 유클리디안 거리로, 위와 같은 방식으로 얼굴을 비교함
##### 하지만 임베딩 모델이 사용되어 좀 더 정확한 얼굴 특징 추출이 가능함
## !랜드마크를 찍은다음 얼굴 임베딩 모델로 특징들을 추출하여 비교!
## !이전 방식은 랜드마크를 찍고 바로 비교를 했었음.!

##### 비교방식은 변하지 않았지만 비교하는 정보들이 좀더 디테일 해짐 

##### 얼굴 임베딩 모델에서의 임계값은 얼굴 간의 유사도를 측정하여 중복 여부를 판단하는 데 사용. 
##### 임베딩 모델은 얼굴 이미지를 고차원 벡터로 변환하여 얼굴의 특징을 나타내는 임베딩 벡터를 생성. 
##### 이 임베딩 벡터는 얼굴의 고유한 특성을 반영, 서로 다른 얼굴 간의 거리를 측정가능.

## "임계값은 두 얼굴 간의 임베딩 벡터 거리를 기준으로 중복 여부를 결정하는 임계값." 
## "일반적으로 두 얼굴 간의 임베딩 벡터 거리가 임계값보다 작으면 두 얼굴을 중복으로 판단. "
##### 이는 얼굴 간의 유사도를 임계값을 기준으로 이진 분류하는 방식. 
##### 임계값을 조정함으로써 중복 여부의 엄격성을 조정 가능.

# 학습된 임베딩 모델 호출, 함수 지정

(내가 학습시킨거 아님. 아직 할줄 모름. 인터넷에 학습된 모델을 다운로드한 것)

### 얼굴 임베딩 모델 : dlib_face_recognition_resnet_model_v1.dat
https://github.com/davisking/dlib-models/blob/master/dlib_face_recognition_resnet_model_v1.dat.bz2

### 얼굴 랜드마크 모델 : shape_predictor_5_face_landmarks.dat
https://github.com/davisking/dlib-models/blob/master/shape_predictor_5_face_landmarks.dat.bz2

In [6]:
# 얼굴 인식기와 임베딩 모델 호출 
detector = dlib.get_frontal_face_detector()


# 얼굴 랜드마크 모델 파일 지정 
sp = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat") # 교수님 제공 (정확도는 얘가 더 높은듯) 
# sp = dlib.shape_predictor('shape_predictor_5_face_landmarks.dat') # 내가 다운                       
### 둘의 차이 : 얼굴당 68개의 랜드마크 추적 
###           다른 하나는 5개의 랜드마크 추적. (당연히 많은 수의 추적을 하는게 좋음)

# 얼굴 임베딩 모델 
facerec = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')   ## 학습된 모델 호출 

# 얼굴 특징 벡터 추출 함수
def extract_face_embedding(image):
    # 이미지에서 얼굴 검출
    face_rects = detector(image, 1)
    # 얼굴이 하나인 경우에만 임베딩 추출
    if len(face_rects) == 1:
        shape = sp(image, face_rects[0])
        embedding = np.array(facerec.compute_face_descriptor(image, shape))
        return embedding
    else:
        return None

    
# 얼굴 중복 검사 함수
def compare_face_features(face_feature1, face_feature2, threshold):
    # 얼굴 특징 벡터의 유클리디안 거리 계산
    distance = np.linalg.norm(face_feature1 - face_feature2)

    # 거리가 임계값보다 작으면 중복으로 판단
    if distance < threshold:
        return True
    else:
        return False

### 중복검사 시작

# 저장된 얼굴 특징 벡터 리스트 초기화
saved_face_features = []

############################################################################################
# 이미지가 저장된 디렉토리 경로 (개인 설정필요)
image_dir = "/Users/PDG/Desktop/face"   ## 경로 설정 "all" 일 경우 얼굴 부분이 아닌 전체화면에서 인식 

############################################################################################

threshold = 0.4   ## 0.4가 최적값인듯   # 임계값 설정 (gpt: 보편적으로 0.5~1.0 사이)

############################################################################################
# 이미지 디렉토리에서 이미지 파일들을 읽어들여 얼굴 검출 및 중복 검사
for filename in os.listdir(image_dir):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):
        image_path = os.path.join(image_dir, filename)
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # 얼굴 검출
        faces = detector(gray)
        
        for face in faces:
            # 얼굴 특징 벡터 추출
            face_feature = extract_face_embedding(image)
            
            if face_feature is not None:
                # 중복 검사
                is_duplicate = False
                for saved_face_feature in saved_face_features:
                    # 기존에 저장된 얼굴 특징과 비교
                    if compare_face_features(face_feature, saved_face_feature, threshold):
                        is_duplicate = True
                        break
                
                if not is_duplicate:
                    saved_face_features.append(face_feature)
                    # 중복되지 않은 얼굴 이미지 출력
                    cv2.imshow("Face Image", image)
                    cv2.waitKey(0)
############################################################################################
# 중복되지 않은 얼굴들의 개수 출력
num_unique_faces = len(saved_face_features)
print("검출된 인원: ", num_unique_faces,"명")

############################################################################################
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)



In [7]:
import os
import cv2

# 저장된 얼굴 특징 벡터 리스트 초기화
saved_face_features = []

############################################################################################
# 이미지가 저장된 디렉토리 경로 (개인 설정 필요)
image_dir = "/Users/PDG/Desktop/face"   ## 경로 설정 "all" 일 경우 얼굴 부분이 아닌 전체 화면에서 인식 

# 저장할 이미지 디렉토리 경로 (개인 설정 필요)
output_dir = "/Users/PDG/Desktop/face/mi"
output_wanted = "/Users/PDG/Desktop/face/wanted"
img_wanted = cv2.imread('/Users/PDG/Desktop/face/wanted_logo/1071.jpg')
############################################################################################

threshold = 0.4   ## 0.4가 최적값인듯   # 임계값 설정 (gpt: 보편적으로 0.5~1.0 사이)

############################################################################################
# 이미지 디렉토리에서 이미지 파일들을 읽어들여 얼굴 검출 및 중복 검사
for filename in os.listdir(image_dir):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):
        image_path = os.path.join(image_dir, filename)
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # 얼굴 검출
        faces = detector(gray)
        
        for face in faces:
            # 얼굴 특징 벡터 추출
            face_feature = extract_face_embedding(image)
            
            if face_feature is not None:
                # 중복 검사
                is_duplicate = False
                for saved_face_feature in saved_face_features:
                    # 기존에 저장된 얼굴 특징과 비교
                    if compare_face_features(face_feature, saved_face_feature, threshold):
                        is_duplicate = True
                        break
                
                if not is_duplicate:
                    saved_face_features.append(face_feature)
                    # 중복되지 않은 얼굴 이미지 저장
                    output_filename = os.path.join(output_dir, f"face_{len(saved_face_features)}.jpg")
                    cv2.imwrite(output_filename, image)
                    
                    # 중복되지 않은 얼굴 이미지와 합성
                    img_out = img_wanted.copy()
                    h, w, c = image.shape
                    # 합성할 이미지 크기를 늘리기 위해 배율 적용
                    scale_factor = 2.5
                    new_h = int(h * scale_factor)
                    new_w = int(w * scale_factor)
                    face_resized = cv2.resize(image, (new_w, new_h))
                    
                    center_y = img_out.shape[0] // 2 + 135  # 얼굴을 아래로 약간 이동
                    center_x = img_out.shape[1] // 2
                    logo_y = center_y - new_h // 2
                    logo_x = center_x - new_w // 2
                    img_out[logo_y:logo_y+new_h, logo_x:logo_x+new_w, :] = face_resized
                    
                    # 얼굴을 흑백으로 변환
                    face_gray = cv2.cvtColor(face_resized, cv2.COLOR_BGR2GRAY)
                    
                    # 합성된 이미지와 흑백 얼굴 이미지를 합성하여 저장
                    output_filename_with_logo = os.path.join(output_wanted, f"wanted_{len(saved_face_features)}.jpg")
                    img_out_gray = img_out.copy()
                    img_out_gray[logo_y:logo_y+new_h, logo_x:logo_x+new_w] = cv2.cvtColor(face_gray, cv2.COLOR_GRAY2BGR)
                    cv2.imwrite(output_filename_with_logo, img_out_gray)
                    
                    # 중복되지 않은 얼굴 이미지 출력
                    cv2.imshow("Face Image", image)
                    cv2.waitKey(1)
############################################################################################
# 중복되지 않은 얼굴들의 개수 출력
num_unique_faces = len(saved_face_features)
print("범인: ", num_unique_faces,"명")

############################################################################################

cv2.destroyAllWindows()
cv2.waitKey(1)


범인:  2 명


-1